In [4]:
import sys
import numpy as np
import argparse
sys.path.insert(1, 'C:\\Users\\ptut0\Documents\\vocal_ambiguity\\cleese_clone\\')
import cleese_stim as cleese
from cleese_stim.engines import PhaseVocoder
import tomli
import json


from IPython.display import Markdown, display, Audio
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os

# Stimulus generation

<mark> Paige: I try to describe in plain English here what this is meant to achieve, from my understanding of your code, fee free to correct </mark>

Stimuli consist of pairs of sentences (e.g. "I have a water bottle in my bag, but I can't fill/feel it") with a target word (e.g. "fill/feel") which makes sense in both case. We are studying how acoustic parameters in the target word, and in the surrounding sentence in general, can sway the listener's interpretation of the target word in one direction or the other. 

The goal of stimulus generation is to randomize three aspects of sound production in each sentence: first, pitch and stretch contours, in the whole sentence; second, spectrum (using cleese's `eq` transform) in the target word only. 

This is achieved by calling `cleese.process_data(engine = PhaseVocoder,...` with custom breakpoint functions that have to be built manually. We first identify visually the start and end time of the target word in each sentence. 
- pitch and stretch bpfs are built by generating 's'=0.1sec. windows in the [0, time_start] period, in the [time_start, time_end] period, and in the [time_end, :] period, and then concatenated. 
- eq bpfs are built by generating 's'=0.1sec. windows in the [time_start, time_end] period alone.

<mark> Paige: I wonder if it would be more appropriate to have only n=1 eq windows in the word, rather than a  number of s=0.1s. Right now, we have a time-varying eq transformation, i.e. about 2 s=0.1 windows with different spectrum transformations, and I don't really know how to analyse this. I have the feeling that it would be clearer to say: here's how to change the spectrum of the word to sway it one way or another, rather that "here's how you have to modify the first part of the word/last part of the word. This is especially true for short, 200ms words like fill/feel; basically we're going to have the first half of /ee/ with a different spectrum than the last part. In longer target words like bruyant/brillant, if using n=1 we're gonna impose the same spectrum constraint to /uy/ than to the final /an/, but that's not necessarily a problem I think, and the only swaying part is /uy/ </mark>

<mark> I'm also uncertain whether we should apply pitch and stretch transformations in the word itself, rather than outside, i.e. only on [0, time_start] and [time_end, :]. On the one hand, it is clear that stretching "fill" will make it sound closer to "feel" - finding this in our data would be a sanity check; on the other hand, I wonder if we are interested in such effects, that are specific to the target word (stretching "fou" doesn't make it sound closer to "fut", and it's not true in general that you should stretch difficult word perhaps). I think we were interested in general facilitating effects of surrounding prosody, i.e. should we e.g. pause before the target word, pause after it, etc. and how they interact with pronunciation within the word itself, so my initial intuition was to not change pitch and stretch within the word (i.e. compare exactly similar words in terms of pitch and stretch). In fact, if we want to transform the inside of the target word, why not just apply the simple s=0.1 throughout, instead of painstakingly concatenate 3 series of s=0.1s bpfs ?!  </mark>  

Because cleese chained and batched transformation (generating n files in one call, each with a chain of transformation) doesn't support custom bpfs (<mark> or does it ?</mark>), we chain transformations manually by passing `wave_out` of one transformation as `wave_in` of the next transformation, and loop over `process_data` to generate many files. 



<mark> Paige: I have a couple of issues with your code: 
- I have the impression that the eq bpf is also transforming [0, time_start] and [time_end, :], instead of just the target word
- as you apply pitch and stretch before eq, I think the boundaries of the word are wrong (because of a preceding stretch); eq should be applied first
- chaining pitch and stretch makes the resulting .json wrong: basically, the pitch bpf is stretched, so the time point does't correspond to times where that pitch transform is actually heard in the final file. This is something that cleese should fix internally, but we can try to fix it here. 
</mark>    
    
    
    

<mark> I don't know what happens in cleese with a bpf that doesn't start at 0 and doesn't end at :, i.e. one that only transforms one window in the target word </mark>

<mark> I also suspect generateTimeVec doesn't work when time points are not consecutive (ex. with ramps) - there's a lot of bug fixing in cleese </mark>

## JJ's code

In [54]:
def load_transform_config(transform, config_file): 
    '''
        returns a dict with the randomization parameters of the transformation
    '''
    with open(config_file, "rb") as f:
        config_pars = tomli.load(f)
        config = config_pars[transform]
    return config 

#test
load_transform_config(transform='stretch',
                      config_file = "./configs/random_stretch_profile.toml")

{'window': {'len': 0.1, 'unit': 's'},
 'std': 1.5,
 'trunc': 1,
 'BPFtype': 'ramp'}

In [65]:
from cleese_stim.engines.phase_vocoder import bpf as cleese_bpf

def generate_custom_bpf_times(target_word_start, target_word_end, total_duration, transform_config):
    ''' 
    This generates a list of time_points outside the target word, 
    by concatenating the output of several calls to PhaseVocoder.create_BPF_time_vec(). 
    The output of this function is meant to be passed to PhaseVocoder.create_BPF 
    
    transform_config is the randomization parameters of the transformation, namely
    with open("./configs/random_stretch_profile.toml", "rb") as f:
        config_pars = tomli.load(f)
        config = config_pars[tr]
    
    '''
            
    # generate time_vec in the [0, word_start] period
    pre_duration = target_word_start
    bpf_time_pre, num_points, end_on_trans = cleese_bpf.createBPFtimeVec(pre_duration,
                                                                  transform_config)
    # generate time_vec in the [word_end, :] period
    post_duration = total_duration - target_word_end
    bpf_time_post, num_points, end_on_trans = cleese_bpf.createBPFtimeVec(post_duration,
                                                                   transform_config)
        
    # generate time_vec in the [word_start, word_end] period
    # NOTE: I QUESTION WHETHER APPROPRIATE
    word_duration = target_word_end - target_word_start
    bpf_time_word, num_points, end_on_trans = cleese_bpf.createBPFtimeVec(word_duration,
                                                                   transform_config)
    
    # create_BPF_time_vec works on a single duration parameter, 
    # and outputs from 0 to duration, so we then correct for the initial times of each period
    bpf_time_word = [bp + target_word_start for bp in bpf_time_word]
    bpf_time_post = [bp + target_word_end for bp in bpf_time_post]
    
    # drop duplicate time between bpfs
    del bpf_time_word[0]
    del bpf_time_post[0]
    
    bpf_time_total = list(bpf_time_pre) + bpf_time_word + bpf_time_post
    
    return bpf_time_total

# test
bpf_times = generate_custom_bpf_times(target_word_start = 2.1, 
                          target_word_end = 2.3, 
                          total_duration = 3, 
                          transform_config = load_transform_config('stretch',
                                                                   "./configs/random_stretch_profile.toml"))
print(bpf_times)
# NOTE FOR SOME REASON, createBPFtimeVec will add 'duration' twice at the end of bpf
    

[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0, 1.1, 1.2000000000000002, 1.3, 1.4000000000000001, 1.5, 1.6, 1.7000000000000002, 1.8, 1.9000000000000001, 2.0, 2.1, 2.1, 2.2, 2.3, 2.4, 2.5, 2.5999999999999996, 2.6999999999999997, 2.8, 2.9, 3.0, 3.0]


In [66]:
bpf_times = generate_custom_bpf_times(target_word_start = 2.1, 
                          target_word_end = 2.3, 
                          total_duration = 3, 
                          transform_config = load_transform_config(transform = 'stretch',
                                                                   config_file = "./configs/random_stretch_profile.toml"))

with open("./configs/random_stretch_profile.toml", "rb") as f:
    config_pars = tomli.load(f)
        
bpf = cleese_bpf.createBPF(tr = 'stretch',
               config = config_pars,
               BPFtime = bpf_times,
               numPoints = len(bpf_times),
               endOnTrans = False,
               eqFreqVec = None)
print(bpf)
  

[[0.         2.44872885]
 [0.1        2.14753947]
 [0.2        2.38982888]
 [0.3        2.12888199]
 [0.4        1.        ]
 [0.5        1.        ]
 [0.6        2.10037288]
 [0.7        2.45888342]
 [0.8        1.48232835]
 [0.9        1.26947323]
 [1.         1.        ]
 [1.1        0.40533927]
 [1.2        1.65572334]
 [1.3        0.26845518]
 [1.4        1.        ]
 [1.5        0.45349286]
 [1.6        1.75437296]
 [1.7        1.3369541 ]
 [1.8        2.35924536]
 [1.9        1.        ]
 [2.         2.47968217]
 [2.1        0.48900782]
 [2.1        1.31026847]
 [2.2        0.241813  ]
 [2.3        0.09608062]
 [2.4        2.04884418]
 [2.5        0.79678745]
 [2.6        1.1806358 ]
 [2.7        1.74462851]
 [2.8        0.7209247 ]
 [2.9        1.82987847]
 [3.         0.19416455]
 [3.         0.88398175]]


## Paige's code

In [33]:
def generate_file(input_file, transforms, time_points, config_dict, output_file, audio_path, bpf_path):

  wave_in, sr, _ = PhaseVocoder.wavRead(input_file)
  duration = len(wave_in) / float(sr)
  
  for key in config_dict:
    f = open(config_dict[key], "rb")
    if key == 'stretch':
      stretch_config = tomli.load(f)
    if key == 'pitch':
      pitch_config = tomli.load(f)
    if key == 'eq':
      eq_config = tomli.load(f)
  
  word = input_file.split("_")[-1]
  word = word.split(".")[0]
  
  time_points_word = np.array(time_points[word]) # values found in audacity
  # phrase before the word
  pre_duration = time_points_word[0]
  # phrase after the word
  post_duration = duration - time_points_word[1]
  word_duration = time_points_word[1] - time_points_word[0]
  
  def generateCustomBP(transform, pre_duration, post_duration, word_duration, config, config_file):
    bpf_time_pre, num_points, end_on_trans = PhaseVocoder.create_BPF_time_vec(
        pre_duration,
        config[transform]
    )
    bpf_time_post, num_points, end_on_trans = PhaseVocoder.create_BPF_time_vec(
        post_duration,
        config[transform]
    )
    bpf_time_word, num_points, end_on_trans = PhaseVocoder.create_BPF_time_vec(
        word_duration,
        config[transform]
    )
    # adjust to correct time points
    bpf_time_post = [bp + time_points_word[1] for bp in bpf_time_post]
    bpf_time_word = [bp + time_points_word[0] for bp in bpf_time_word]
    bpf_time_pre = list(bpf_time_pre)
    
    # drop duplicate time between bpfs
    del bpf_time_post[0]
    del bpf_time_pre[-1]
    
    full_time_points = bpf_time_pre + bpf_time_word + bpf_time_post
  
    num_points = len(full_time_points)
    
    eqFreqVec = None
    if transform == 'eq':
      eqFreqVec = PhaseVocoder.createBPFfreqs(config)
  
    # create a bpf for each of the transformations
    bpf_list = (PhaseVocoder.create_BPF(
        transform,
        config_file,
        full_time_points,
        num_points,
        end_on_trans,
        eqFreqVec
    ))
  
    return bpf_list
  
  bpf_dict = {}

  if 'stretch' in transforms:
    bpf_list = generateCustomBP('stretch', pre_duration, post_duration, word_duration, stretch_config, config_dict['stretch'])
    wave_out,bpf_out = cleese.process_data(
        PhaseVocoder,
        wave_in,
        config_dict['stretch'],
        sample_rate=sr,
        BPF=bpf_list
    )
    wave_in = wave_out
    bpf_dict['stretch'] = str(bpf_out)
  
  if 'pitch' in transforms:
    bpf_list = generateCustomBP('pitch', pre_duration, post_duration, word_duration, pitch_config, config_dict['pitch'])
    wave_out,bpf_out = cleese.process_data(
        PhaseVocoder,
        wave_in,
        config_dict['pitch'],
        sample_rate=sr,
        BPF=bpf_list
    )
    wave_in = wave_out
    bpf_dict['pitch'] = str(bpf_out)
  
  if 'eq' in transforms:
    bpf_list = generateCustomBP('eq', pre_duration, post_duration, word_duration, eq_config, config_dict['eq'])
    wave_out,bpf_out = cleese.process_data(
        PhaseVocoder,
        wave_in,
        config_dict['eq'],
        sample_rate=sr,
        BPF=bpf_list
    )
    wave_in = wave_out
    bpf_dict['eq'] = str(bpf_out)

  PhaseVocoder.wavWrite(wave_out, audio_path+output_file+'.wav', sr)
  with open(bpf_path + output_file+'.json', 'w') as f:
    json.dump(bpf_dict, f)


In [35]:
parent_dir = "C:\\Users\\ptut0\\Documents\\vocal_ambiguity\\stimulus_generation"

input_file = "./sounds/bottle_feel.wav"

config_dict = {
  'stretch': "./configs/random_stretch_profile.toml",
  'pitch': "./configs/random_pitch_profile.toml",
  'eq': "./configs/random_timbre_profile.toml"
}

transforms = ['stretch', 'pitch', 'eq']

num_files = 50

#The timepoints for each of the words of intrest for each phrase
time_points = {
  "feel" : [2.51, 2.71],
  "fill" : [2.32, 2.51],
  "beat" : [2.51, 2.69],
  "bit" : [2.41, 2.53],
  "fou" : [],
  "fut" : [0.23, 0.51],
  "brillant" : [2.49, 2.90],
  "bruyant" : [2.42, 2.90]
}

base_file = os.path.splitext(os.path.basename(input_file))[0]
file_path = os.path.join(parent_dir, base_file)
audio_path = os.path.join(parent_dir, base_file, 'audio\\') 
bpf_path = os.path.join(parent_dir, base_file, 'bpf\\')
if not os.path.exists(file_path):
  os.mkdir(file_path)
  os.mkdir(audio_path)
  os.mkdir(bpf_path)



for i in range(num_files):
  output_file  = base_file + f'_{i+1}'
  generate_file(input_file, transforms, time_points, config_dict, output_file, audio_path, bpf_path)

WARN: stereo file detected. Reading only left channel.
stretch variation 1/1
pitch variation 1/1
eq variation 1/1
WARN: stereo file detected. Reading only left channel.
stretch variation 1/1


{'stretch': '[[0.         0.21238429]\n [0.1        0.42781447]\n [0.2        1.36223402]\n [0.3        0.8909512 ]\n [0.4        1.18738259]\n [0.5        0.29685975]\n [0.6        1.60814124]\n [0.7        0.78354962]\n [0.8        1.13353641]\n [0.9        1.        ]\n [1.         1.03352626]\n [1.1        0.21778415]\n [1.2        0.39644294]\n [1.3        0.71431735]\n [1.4        0.1540186 ]\n [1.5        1.80566125]\n [1.6        0.80694543]\n [1.7        0.9920155 ]\n [1.8        1.0280846 ]\n [1.9        1.96208122]\n [2.         0.8204204 ]\n [2.1        0.18849739]\n [2.2        1.38351309]\n [2.3        1.        ]\n [2.4        1.46052404]\n [2.5        1.        ]\n [2.51       0.71349925]\n [2.61       0.4055214 ]\n [2.71       1.82103505]\n [2.71       0.68813516]\n [2.81       2.31684206]\n [2.91       1.78502033]\n [2.95909297 0.27972625]]', 'pitch': '[[ 0.00000000e+00 -1.23027199e+02]\n [ 1.00000000e-01  9.46933685e+01]\n [ 2.00000000e-01 -5.75435234e+01]\n [ 3.0000

pitch variation 1/1
eq variation 1/1
WARN: stereo file detected. Reading only left channel.
stretch variation 1/1


{'stretch': '[[0.         1.07443877]\n [0.1        2.36558799]\n [0.2        1.        ]\n [0.3        0.10110521]\n [0.4        2.31918142]\n [0.5        1.        ]\n [0.6        1.64930359]\n [0.7        0.68388132]\n [0.8        1.56519623]\n [0.9        2.45239905]\n [1.         1.74723791]\n [1.1        0.90064208]\n [1.2        1.68542572]\n [1.3        0.18896808]\n [1.4        1.12060514]\n [1.5        0.68899814]\n [1.6        1.52939519]\n [1.7        1.04880906]\n [1.8        2.28095545]\n [1.9        1.37466728]\n [2.         2.36008554]\n [2.1        1.        ]\n [2.2        1.61641215]\n [2.3        0.70889322]\n [2.4        2.46306819]\n [2.5        0.69425505]\n [2.51       0.164326  ]\n [2.61       1.89299551]\n [2.71       1.66976724]\n [2.71       1.79999772]\n [2.81       0.84772937]\n [2.91       2.15590431]\n [2.95909297 1.        ]]', 'pitch': '[[ 0.00000000e+00 -2.36328759e+00]\n [ 1.00000000e-01  9.87419599e+01]\n [ 2.00000000e-01  1.43283401e+02]\n [ 3.0000

pitch variation 1/1
eq variation 1/1
WARN: stereo file detected. Reading only left channel.
stretch variation 1/1


...

WARN: stereo file detected. Reading only left channel.
stretch variation 1/1
pitch variation 1/1
eq variation 1/1


{'stretch': '[[0.         1.13993749]\n [0.1        1.93616987]\n [0.2        0.03240805]\n [0.3        0.00865107]\n [0.4        0.59586596]\n [0.5        1.        ]\n [0.6        2.01790496]\n [0.7        1.60835508]\n [0.8        1.32364362]\n [0.9        1.8048555 ]\n [1.         1.        ]\n [1.1        2.10358427]\n [1.2        0.19475947]\n [1.3        0.59591554]\n [1.4        0.93983692]\n [1.5        1.59896859]\n [1.6        0.37584703]\n [1.7        0.84248003]\n [1.8        1.04592718]\n [1.9        1.56766737]\n [2.         0.46113607]\n [2.1        0.17504251]\n [2.2        0.98191708]\n [2.3        1.50325271]\n [2.4        1.        ]\n [2.5        0.08072663]\n [2.51       1.        ]\n [2.61       0.93888258]\n [2.71       0.26992119]\n [2.71       1.14288596]\n [2.81       2.01566864]\n [2.91       1.42142563]\n [2.95909297 0.89437949]]', 'pitch': '[[ 0.00000000e+00  4.35547765e+01]\n [ 1.00000000e-01 -1.06137146e+02]\n [ 2.00000000e-01  1.26105049e+02]\n [ 3.0000

WARN: stereo file detected. Reading only left channel.
stretch variation 1/1
pitch variation 1/1
eq variation 1/1


{'stretch': '[[0.         1.00084022]\n [0.1        0.56855978]\n [0.2        0.17507357]\n [0.3        0.3547358 ]\n [0.4        0.20051873]\n [0.5        0.80529349]\n [0.6        0.96853813]\n [0.7        1.        ]\n [0.8        0.32514168]\n [0.9        0.68029601]\n [1.         2.2132618 ]\n [1.1        1.05029795]\n [1.2        2.12485354]\n [1.3        0.64208916]\n [1.4        0.82143717]\n [1.5        2.17151663]\n [1.6        0.75183411]\n [1.7        2.15615754]\n [1.8        1.86117181]\n [1.9        2.05752046]\n [2.         0.31071489]\n [2.1        1.433263  ]\n [2.2        1.23186494]\n [2.3        1.95283685]\n [2.4        1.        ]\n [2.5        1.        ]\n [2.51       0.49564402]\n [2.61       0.8221671 ]\n [2.71       1.01922292]\n [2.71       1.99597852]\n [2.81       0.34604364]\n [2.91       1.53085482]\n [2.95909297 1.        ]]', 'pitch': '[[ 0.00000000e+00  2.98410387e+01]\n [ 1.00000000e-01 -8.20500808e+01]\n [ 2.00000000e-01  1.11099781e+02]\n [ 3.0000

WARN: stereo file detected. Reading only left channel.
stretch variation 1/1
